In [ ]:
#@title ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title データを保存するディレクトリのパスを指定
#@markdown ランタイムに接続しドライブを後、コピーしてくる<br>
#@markdown 後ろの / は追加するので、追記は不要<br>

cwd = "/content/drive/MyDrive/RNA-seq/Outputs/\u5B9F\u9A13\u540D" #@param {type:"string"}
cwd = cwd + "/"

# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

# カレントディレクトリの変更
import os
os.makedirs(cwd , exist_ok = True)
os.chdir(cwd)
currentdir = os.getcwd()

In [ ]:
#@title 実験名を定義
exp_name = "\u5B9F\u9A13\u540D" #@param {type:"string"}
exp_name = exp_name + "_"

In [ ]:
#@title DEGと平均行を追加したTPMデータのパスを定義
df_tpm_path =  "/content/drive/MyDrive/RNA-seq/Outputs/Hypoxia2/Hypoxia2_DEGavetpm.csv" #@param {type:"string"}
df_tpm = pd.read_csv(df_tpm_path , index_col = 0)
df_tpm

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
読み込んだデータから、必要なデータを抽出する

In [ ]:
#@markdown 探すものの名前をカンマ区切りで入力<br>正規表現可、詳細は下のページを参考に<br>https://qiita.com/syuki-read/items/6c5f139abd676fcc0ca2
#@markdown 
drop_things = "" #@param {type:"string"}
name = drop_things
drop_things = drop_things.replace(' ', '')
drop_things = drop_things.split(',')

df_tyuusyutu = pd.DataFrame()

print("検索ワード:")
for i in range(len(drop_things)):
  print(drop_things[i], end="   ")
  df_tmp = df_tpm[df_tpm["product"].str.contains(drop_things[i] , regex=True)]
  df_tmp = df_tmp.sort_values("product")
  df_tyuusyutu = pd.concat([df_tyuusyutu, df_tmp])

df_tyuusyutu = df_tyuusyutu.drop_duplicates(subset='product')

# printして確認
print("\n=====================================")
print(df_tyuusyutu.to_string(index= False))

In [ ]:
#@title csvファイル形式で抽出してきた遺伝子を保存
df_tyuusyutu.to_csv(name + ".csv" , index = False)

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
グラフを描画する

In [ ]:
#@title 抽出した遺伝子のDEG_log2値をグラフ化
plt.figure(facecolor = "white" , figsize=(10, 5))
sns.barplot(x="DEG_log2" , y="product" , data = df_tyuusyutu)

In [ ]:
#@title N数の定義と必要データを纏めたdfの作成
n_num =  #@param {type:"integer"}
df = df_tyuusyutu.copy()

df

In [ ]:
#@title 平均値にエラーバー(標準誤差)をつけたグラフを描画
df_e = df.set_index("product")
df_e.drop(["DEG" , "DEG_log2"] , axis = 1 , inplace = True)

for i in range(len(df_e)):
  if df_e.iloc[i , :].sum(axis = 0) == 0:
    df_e.drop(df_e.index[i] , axis = 0 , inplace = True)

x = list(df_e.index)
print(x)

df_ave = df_e.iloc[: , [n_num , 2 * n_num + 1]]
df_e.drop(list(df_ave) , axis = 1 , inplace = True)
a = df_e.to_numpy().reshape(len(x) * 2 , n_num)

index_name = []
for gene in x:
  index_name = index_name + [gene + "_Cont" , gene + "_Test"]

y_mean = []
for i in range(len(df_ave)):
  y_mean = y_mean + [df_ave.iloc[i , 0] , df_ave.iloc[i , 1]]

df_e2 = pd.DataFrame(a , index = index_name ,columns = np.arange(n_num) +1)
y_sd = df_e2.std(1) / np.sqrt(n_num)
error_bar_options = {"lw":1 , "capthick":1 , "capsize": 20}

fig = plt.figure(facecolor = "white" , figsize=(len(x) * 4 , len(x)))
ax = fig.add_subplot()
ax.bar(index_name , y_mean , yerr = y_sd , error_kw = error_bar_options)

In [ ]:
#@title 平均を使わず、各データの値を比較
a = df_e.to_numpy().reshape(len(x) * 2 * n_num , 1)
b = list(df_e)

index_name = []
hue_name = []
for i in x:
  hue_name = hue_name + b
  for j in range(n_num * 2):
    index_name = index_name + [i]

fig = plt.figure(facecolor = "white" , figsize=(len(x) * 4 , len(x)))
ax = fig.add_subplot()
a = a.flatten()
sns.barplot(x = index_name , y = a , hue = hue_name)